# 练习题解答

将第一天的项目更新，使用本地运行的开源模型 Ollama 而非 OpenAI 来总结网页内容。

如果您不想使用付费 API，可以在所有后续项目中使用此技术。

**优点：**
1. 无 API 费用 - 开源
2. 数据不会离开您的设备

**缺点：**
1. 性能远不如前沿模型

## Ollama 安装回顾

只需访问 [ollama.com](https://ollama.com) 并安装即可！

安装完成后，ollama 服务器应该已经在本地运行。
如果您访问：
[http://localhost:11434/](http://localhost:11434/)

您应该会看到消息 `Ollama is running`。

如果不是，请打开一个新的终端（Mac）或 Powershell（Windows），输入 `ollama serve`
然后再次尝试访问 [http://localhost:11434/](http://localhost:11434/)。

In [ ]:
# 导入库

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

In [ ]:
# 常量

MODEL = "llama3.2"

In [ ]:
# 一个表示网页的类

class Website:
    """
    一个用于表示我们已抓取的网站的工具类
    """
    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        使用 BeautifulSoup 库从给定的 url 创建此 Website 对象
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "未找到标题"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
# 让我们试一个

ed = Website("https://www.baidu.com")
print(ed.title)
print(ed.text)

## 提示类型

您可能已经知道:

像 GPT4o 这样的模型已经被训练成以特定方式接收指令。

它们期望接收：

**系统提示**，告诉它们正在执行什么任务以及应该使用什么语气

**用户提示** -- 它们应该回复的对话启动器

In [ ]:
# 定义我们的系统提示 - 您稍后可以对此进行实验，将最后一句话更改为 'Respond in markdown in english.'

system_prompt = "您是一个分析网站内容并提供简短摘要的助手，忽略可能与导航相关的文本。\
以 markdown 格式回复。"

In [ ]:
# 一个编写用户提示的函数，用于请求网站摘要：

def user_prompt_for(website):
    user_prompt = f"您正在查看一个标题为 {website.title} 的网站"
    user_prompt += "该网站的内容如下；\
请以 markdown 格式提供该网站的简短摘要。\
如果包含新闻或公告，也请一并总结。\n\n"
    user_prompt += website.text
    return user_prompt

## 消息

Ollama 的 API 期望与 OpenAI 相同的消息格式：

```
[
    {"role": "system", "content": "系统消息在此"},
    {"role": "user", "content": "用户消息在此"}
]
```

In [ ]:
# 看看这个函数如何精确地创建上述格式

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

## 是时候整合起来了 - 现在使用 Ollama 而非 OpenAI

In [ ]:
# 现在：调用 Ollama 函数而不是 OpenAI

def summarize(url):
    website = Website(url)
    messages = messages_for(website)
    response = ollama.chat(model=MODEL, messages=messages)
    return response['message']['content']

In [ ]:
summarize("https://51cto.com")

In [ ]:
# 一个函数，用于在 Jupyter 输出中以 markdown 格式漂亮地显示摘要

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://51cto.com")

# 让我们尝试更多网站

请注意，这仅适用于可以使用这种简单方法抓取的网站。

使用 Javascript 渲染的网站，如 React 应用，将不会显示。您需要阅读有关安装 Selenium 的信息（询问 ChatGPT！）

此外，受 CloudFront（及类似服务）保护的网站可能会出现 403 错误。

但许多网站都可以正常工作！

In [ ]:
display_summary("https://ollama.com")

In [ ]:
display_summary("https://anthropic.com")